In [2]:
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
 
def calculate_fid(act1, act2):
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis= 0), cov(act1, rowvar= False)
    mu2, sigma2 = act2.mean(axis= 0), cov(act2, rowvar= False)
 
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)** 2.0)
 
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
 
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
        # calculate score
        fid = ssdiff + trace(sigma1 + sigma2 - 2.0*covmean)
    return fid
 
act1 = random(20480)
act1 = act1.reshape((10, 2048))
act2 = random(20480)
act2 = act2.reshape((10, 2048))
fid = calculate_fid(act1, act1)
print( 'FID (same): %.3f'% fid)
fid = calculate_fid(act1, act2)
print( 'FID (different): %.3f'% fid)

FID (same): -0.000
FID (different): 355.916


In [3]:
a = 10000
c = 28000
f = [a // 5] * c
b, e = 0, 0

while b < c:
    d = 0
    g = c * 2
    b = c

    while b > 0:
        d = d * b + a * f[b - 1]
        f[b - 1] = d % g
        d //= g
        g -= 1
        b -= 1

    c -= 14
    print(f"{e + d // a:04d}", end='')
    e = d % a

print()


0000071428585724295872416264232124958788726698765257487855733699467938126443339988512296537067817423947296173600376511482180184191489520906383948558342595335131061630379013345177334114081256879665716115347444688998071282559536063167254687200235519513998383458883441559247082329470571132664695253037956857291354388183641965248758929055340384529674957109270181507774538668697152590905688806899609849115217178969930038386316744273803243326541679292892668532120157468641136797846976749663553115509774836897837347809955815730311521734150424441747651202068209692508516197132872973220052311884281864694472323358802625496410792017953879210334694987591448773293715917932966853873142235211712421003903694498285245266818008302170715072566405026313059910376025974562958006149562233624733881208005062524746598526557825020635273605596629653227222487929264990029529146374241106406978791960315683020694687002328941048320463581534871485246720981436889334197591085633348720849229776912987062455713066140579568903732378

In [5]:
from pathlib import Path
import subprocess
from tqdm import tqdm

def merge_audio_video(path):
    """
    遍历给定路径下所有文件, 如果同一路径下同时存在名为 audio.m4s 和 video.m4s 的两个文件, 
    则将 video.m4s 视为视频, 将 audio.m4s 视为音频进行合并
    """
    path = Path(path)  # 将路径转换为 Path 对象
    for file in tqdm(path.rglob("*.m4s")):  # 使用 rglob 遍历所有 .m4s 文件
        if not file.name.startswith("video"):  # 处理视频文件
            continue
        audio_file = file.with_name(file.name.replace("video", "audio"))  # 获取对应的音频文件
        if not audio_file.exists():  # 如果音频文件存在
            continue
        output_file = file.with_suffix(".mp4")  # 输出文件的路径
        # 使用 ffmpeg 合并音频和视频
        subprocess.call(f'ffmpeg -i "{file}" -i "{audio_file}" -c copy "{output_file}"', shell=True)
        # 删除原始的音频和视频文件
        file.unlink()
        audio_file.unlink()

path = r'F:\视频\HHH\download'
merge_audio_video(path)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 14.19it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 19.41it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 17.49it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.09it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.77it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 13.23it/s]
0it [00:

In [77]:
# 写两个函数, 一个用于加密, 一个用于解密. 加密函数输入文件路径和密钥(字符串), 将文件加密后保存在原路径下, 文件名为"{原文件名}(加密).SEC"; 解密函数输入文件路径和密钥(字符串), 将文件解密后保存在原路径下, 文件名为"{原文件名}(解密).SEC"


import os
from cryptography.fernet import Fernet
import hashlib
import base64

def generate_key_from_string(s):
    # 使用 SHA-256 哈希算法生成 32 字节的哈希值
    hash_value = hashlib.sha256(s.encode()).digest()

    # 使用 base64 编码将哈希值转换为符合 Fernet 要求的密钥
    key = base64.urlsafe_b64encode(hash_value)

    return key

def encrypt(data, key = None):
    """
    使用提供的密钥加密给定的数据并返回加密结果。
    """
    key = key if key != None else Fernet.generate_key()
    f = Fernet(key)
    encrypted_data = f.encrypt(data)
    return encrypted_data, key

def decrypt(data, key):
    """
    使用提供的密钥加密给定的数据并返回加密结果。
    """
    f = Fernet(key)
    decrypted_data = f.decrypt(data)
    return decrypted_data


def encrypt_file(file_path, key):
    """
    Encrypts a file using a given key and saves it in the same directory with the suffix "(加密).SEC"
    """
    with open(file_path, 'rb') as f:
        data = f.read()
        
    key = generate_key_from_string(str(key))
    encrypted_data, key = encrypt(data, key) # replace encrypt with your encryption function
    file_name, file_ext = os.path.splitext(file_path)
    print(key)

    new_file_path = f"{file_name}(加密){file_ext}"
    with open(new_file_path, 'wb') as f:
        f.write(encrypted_data)

def decrypt_file(file_path, key):
    """
    Decrypts a file using a given key and saves it in the same directory with the suffix "(解密).SEC"
    """
    with open(file_path, 'rb') as f:
        data = f.read()
    
    key = generate_key_from_string(str(key))
    print(key)
    decrypted_data = decrypt(data, key) # replace decrypt with your decryption function
    file_name, file_ext = os.path.splitext(file_path)
    new_file_path = f"{file_name.split('(加密)')[0]}(解密){file_ext}"
    with open(new_file_path, 'wb') as f:
        f.write(decrypted_data)

# encrypt_file(r'F:\图片\Camera Roll\1555906472520.gif', '13')
decrypt_file(r'F:\图片\Camera Roll\1555906472520(加密).gif', '13')


b'P9ujXwTcjEYphsmSvPh1VGJXETByqQnBYvfkcOWB4ng='


In [70]:
decrypt_file(r'F:\图片\Camera Roll\1555906472520(加密).SEC', b'vo03-siQI05fcNU0VMhzanti-kVdKC2JJs2t9C0HRB8=')

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\图片\\Camera Roll\\1555906472520(加密).SEC'

In [2]:
import hashlib
import base64

def generate_key_from_string(s):
    # 使用 SHA-256 哈希算法生成 32 字节的哈希值
    hash_value = hashlib.sha256(s.encode()).digest()

    # 使用 base64 编码将哈希值转换为符合 Fernet 要求的密钥
    key = base64.urlsafe_b64encode(hash_value)

    return key

# 将字符串 "13" 转换为符合要求的密钥
key = generate_key_from_string("13")

print("Generated key:", key)


Generated key: b'P9ujXwTcjEYphsmSvPh1VGJXETByqQnBYvfkcOWB4ng='


In [1]:
from sklearn.metrics import roc_auc_score

# 假设您已经获得了预测概率和真实标签
# 这里只是一个示例，您需要根据自己的数据进行替换
y_true = [0, 1, 1, 0, 1, 0, 1]
y_pred_prob = [0.1, 0.9, 0.8, 0.2, 0.7, 0.3, 0.95]

# 使用roc_auc_score计算AUC值
auc = roc_auc_score(y_true, y_pred_prob)
print("AUC值为：", auc)


ModuleNotFoundError: No module named 'sklearn'

In [6]:
import re

def convert_chapter_numbers(text):
    # 将找到的数字转换为"第{数字}章"
    result = re.sub(pattern, lambda m: '第' + m.group(1) + '章 ', text)

    return result

# 映射阿拉伯数字到中文数字
num_dict = {
    '0': '零', '1': '一', '2': '二', '3': '三', '4': '四',
    '5': '五', '6': '六', '7': '七', '8': '八', '9': '九',
    '10': '十', '100': '百', '1000': '千', '10000': '万'
}

def convert_to_chinese_num(match):
    num_str = match.group(1)
    result = ''
    for digit in num_str:
        result += num_dict[digit]
    return '第' + result + '章\n'

def replace_numbers(text):
    pattern = r'第(\d+)章'
    result = re.sub(pattern, convert_to_chinese_num, text)
    return result

# 正则表达式寻找"{数字}. "的形式
pattern = r'（(\d+)）\s'
path = r'F:\书籍\小说\轻小说\妹控\我和我的妹妹雯雯\新建文件夹\我和我的妹妹雯雯.txt'
with open(path, 'r', encoding = 'utf-8') as f:
    file = f.read()
    
text = convert_chapter_numbers(file)
# text = replace_numbers(text)
with open(path, 'w', encoding = 'utf-8') as f:
    f.write(text)

In [17]:
from pdf2docx import Converter

def convert_pdf_to_word(pdf_file_path):
    # 创建转换器对象
    cv = Converter(pdf_file_path)

    # 转换pdf到word并保存
    cv.convert(pdf_file_path.replace('pdf', 'docx'), start=0, end=None)
    cv.close()

    
convert_pdf_to_word(r'G:\Document\My Document\科研助理\验收\面向医学影像-项目验收材料\8 资金情况证明材料\2023\住宿费_2619\学术会议通知.pdf')

[INFO] Start to convert G:\Document\My Document\科研助理\验收\面向医学影像-项目验收材料\8 资金情况证明材料\2023\住宿费_2619\学术会议通知.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[WARNING] Words count: 0. It might be a scanned pdf, which is not supported yet.
[INFO] [3/4] Parsing pages...
[INFO] (1/3) Page 1
[INFO] (2/3) Page 2
[INFO] (3/3) Page 3
[INFO] [4/4] Creating pages...
[INFO] (1/3) Page 1
[INFO] (2/3) Page 2
[INFO] (3/3) Page 3
[INFO] Terminated in 1.31s.


In [4]:
import os
from docx import Document

def merge_docx_files_in_dir(dir_path, output_filename):
    # 获取目录下所有.docx文件列表
    docx_files = [f for f in os.listdir(dir_path) if f.endswith('.docx')]

    # 按文件名排序
    docx_files.sort()

    # 创建一个新的Word文档用于保存合并后的内容
    merged_document = Document()

    # 遍历每个.docx文件
    for docx_file in docx_files:
        sub_doc = Document(os.path.join(dir_path, docx_file))

        # 将每个子文档的内容添加到合并的文档中
        for element in sub_doc.element.body:
            merged_document.element.body.append(element)

    # 保存合并的文档
    merged_document.save(os.path.join(dir_path, output_filename))

    print(f'Merged {len(docx_files)} docx files into {output_filename}')

    
merge_docx_files_in_dir(r'F:\文档\My Document\科研助理\验收\报告\最终报告\529', 'merge.docx')

Merged 2 docx files into merge.docx


In [ ]:
import cv2
import numpy as np

# 加载视频
video = cv2.VideoCapture('path_to_video.mp4')

# 检查视频是否成功加载
if not video.isOpened():
    print("无法加载视频")
    exit()

# 获取视频的帧率和尺寸
fps = video.get(cv2.CAP_PROP_FPS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 创建VideoWriter对象，用于保存处理后的视频
output = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 循环读取视频的每一帧
while True:
    ret, frame = video.read()

    # 检查是否成功读取帧
    if not ret:
        break

    # 创建一个与视频帧相同大小的遮罩（mask），并将其初始化为全黑
    mask = np.zeros((height, width), dtype=np.uint8)

    # 定义模糊区域的顶点坐标
    top_right = (int(width * 0.75), int(height * 0.25))
    bottom_right = (width, 0)

    # 在遮罩上绘制一个矩形，表示要模糊的区域
    cv2.rectangle(mask, top_right, bottom_right, (255, 255, 255), -1)

    # 将遮罩应用于视频帧，只对指定区域进行模糊处理
    blurred_frame = cv2.GaussianBlur(frame, (25, 25), 0)
    blurred_frame = cv2.bitwise_and(blurred_frame, blurred_frame, mask=mask)

    # 将模糊的帧与原始帧合并
    result_frame = cv2.add(frame, blurred_frame)

    # 将合并后的帧写入输出视频
    output.write(result_frame)

    # 显示合并后的帧
    cv2.imshow('Blurred Frame', result_frame)

    # 按下 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
video.release()
output.release()
cv2.destroyAllWindows()


In [ ]:
import os
import subprocess

def merge_videos(video_dir, output_file):
    """
    Merge all videos in the specified directory into a single video file.
    
    :param video_dir: Directory containing video files
    :param output_file: Name of the output video file
    """
    videos = [f for f in os.listdir(video_dir) if f.endswith('.mov')]
    with open('files.txt', 'w') as f:
        for video in videos:
            f.write(f"file '{video_dir}/{video}'\n")
    
    command = "ffmpeg -f concat -safe 0 -i files.txt -c copy " + f'"{video_dir}/{output_file}"'
    print(command)
    subprocess.call(command, shell=True)

merge_videos(r'Z:\temp\杨清武 基金申报与标书写作', '杨清武 基金申报与标书写作.mp4')


ffmpeg -f concat -safe 0 -i files.txt -c copy "Z:\temp\杨清武 基金申报与标书写作/杨清武 基金申报与标书写作.mp4"


In [22]:
import cv2
import numpy as np

def repair_image(img_path):
    # 读取图片
    img = cv2.imread(img_path, 0)

    # 应用中值滤波器
    repaired_img = cv2.medianBlur(img, 5)

    # 显示原始图片和修复后的图片
    cv2.imshow('Original Image', img)
    cv2.imshow('Repaired Image', repaired_img)

    # 等待关闭窗口
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# 使用函数
repair_image(r'Z:\Photo\Saved Pictures_re\微信图片_20230726191025.jpg')


In [5]:
import os
import shlex
import subprocess

def concat_and_convert_ts_to_mp4(ts_files_dir, output_mp4_path):
    """
    Concatenate a series of .ts files and convert to .mp4 using ffmpeg.
    
    :param ts_files_dir: Directory containing .ts files named as video001.ts, video002.ts, ...
    :param output_mp4_path: Destination path for the .mp4 file
    """

    output_mp4_path = ts_files_dir + '.mp4'

    # 获取目录下的所有.ts文件，并按名称排序
    ts_files = sorted([f for f in os.listdir(ts_files_dir) if f.endswith('.ts')])
    # ts_files_path = '|'.join([os.path.join(ts_files_dir, ts_file) for ts_file in ts_files])
    ts_files_path = '|'.join([shlex.quote(os.path.join(ts_files_dir, ts_file)) for ts_file in ts_files])
    
    concat_cmd = f'concat:{ts_files_path}'
    
    command = [
        'ffmpeg',
        '-i', concat_cmd,
        '-c:v', 'libx264',
        '-c:a', 'aac',
        '-strict', 'experimental',
        '-f', 'mp4',
        output_mp4_path
    ]

    subprocess.run(command)

if __name__ == "__main__":
    # 输入目录和输出路径
    input_dir = r'F:\下载\新建文件夹'
    
    concat_and_convert_ts_to_mp4(input_dir)
    print("Conversion finished!")


Conversion finished!


In [5]:
with open(r'F:\下载\2023092118231391537.jpg', 'rb') as f:
    encrypt_data = f.read()

In [17]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import base64

def decrypt(encrypted_data, key, iv):
    # Create AES cipher object
    cipher = AES.new(key.encode(), AES.MODE_CBC, iv.encode())
    
    # Decrypt the data
    decrypted_data = cipher.decrypt(encrypted_data)
    
    # Unpad the decrypted data
    decrypted_data = unpad(decrypted_data, AES.block_size)
    
    return decrypted_data

def convert_words_to_bytes(words):
    # Convert each 32-bit word to 4 bytes
    bytes_list = [word.to_bytes(4, byteorder='big') for word in words]
    # Concatenate all bytes
    return b''.join(bytes_list)

# Words arrays from the JavaScript environment
words_key = [1714775097, 909468774, 926233395, 909260592]
words_iv = [959930934, 808663348, 1633837874, 1717724465]

# Convert words to bytes
key_bytes = convert_words_to_bytes(words_key)
iv_bytes = convert_words_to_bytes(words_iv)

# Convert bytes to hexadecimal and decode to UTF-8
key_string = bytes.fromhex(key_bytes.hex()).decode('utf-8')
iv_string = bytes.fromhex(iv_bytes.hex()).decode('utf-8')

# Load the encrypted data (assuming it's stored in a file called "encrypted_image.jpg")
with open(r'F:\下载\2023092118231391537.jpg', "rb") as file:
    encrypted_data_bytes = file.read()

# Decrypt the data
decrypted_data = decrypt(encrypted_data_bytes, key_string, iv_string)

# Save the decrypted data to a new file
with open(r'F:\下载\decrypted_image.jpg', "wb") as file:
    file.write(decrypted_data)



In [19]:
key_string, iv_string

('f5d965df75336270', '97b60394abc2fbe1')

In [ ]:
import os
import mimetypes
import magic
from pathlib import Path

def check_file_types_in_folder(folder_path: Path):
    """
    遍历指定文件夹，检查文件扩展名与实际文件类型是否一致，
    如果不一致则打印出来。

    :param folder_path: 文件夹的 Path 对象。
    """
    # 确保输入是一个有效的文件夹
    if not folder_path.is_dir():
        raise ValueError(f"{folder_path} 不是有效的文件夹")

    # 创建 Magic 对象，用于根据文件内容推断 MIME 类型
    file_magic = magic.Magic(mime=True)

    # 遍历文件夹及其子文件夹中的所有文件
    for file_path in folder_path.rglob('*'):  # rglob('*') 会遍历所有子文件夹
        if file_path.is_file():
            # 获取文件的扩展名
            ext = file_path.suffix.lower()
            
            # 使用 mimetypes 判断扩展名的 MIME 类型
            guessed_mime, _ = mimetypes.guess_type(file_path)
            
            # 使用 magic 判断文件内容的 MIME 类型
            actual_mime = file_magic.from_file(file_path)

            # 打印扩展名和实际 MIME 类型不匹配的文件
            if guessed_mime != actual_mime:
                print(f"文件: {file_path}")
                print(f"  扩展名推测的 MIME 类型: {guessed_mime}")
                print(f"  实际 MIME 类型: {actual_mime}")
                print("-" * 50)

# 示例使用
folder_path = Path('/path/to/your/folder')  # 将此路径替换为你想检查的文件夹路径
check_file_types_in_folder(folder_path)


In [1]:
import tkinter as tk
from tkinter import messagebox
import os
import base64, json
from Cryptodome.Cipher import AES, PKCS1_OAEP
from Cryptodome.PublicKey import RSA

class SecureTextApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AES + RSA Secure Text Encryptor")
        self.root.geometry("800x500")

        # 生成 RSA 密钥对
        self.rsa_key = RSA.generate(2048)
        self.public_key = self.rsa_key.publickey()
        self.private_key = self.rsa_key

        # UI 布局
        self.create_widgets()

    def create_widgets(self):
        # 加密方式选项
        self.mode_label = tk.Label(self.root, text="加密方式", font=("Arial", 12))
        self.mode_label.grid(row=0, column=0, columnspan=3, pady=5)
        
        self.encryption_mode = tk.StringVar(value="AES+RSA")
        self.mode_menu = tk.OptionMenu(self.root, self.encryption_mode, "AES", "RSA", "AES+RSA", command=self.update_key_fields)
        self.mode_menu.grid(row=1, column=0, columnspan=3, pady=5)

        # 左侧 - 源文本
        self.src_label = tk.Label(self.root, text="源文本", font=("Arial", 12))
        self.src_label.grid(row=2, column=0, padx=10, pady=5)
        self.src_text = tk.Text(self.root, width=40, height=10)
        self.src_text.grid(row=3, column=0, padx=10, pady=5)

        # 右侧 - 加密文本
        self.enc_label = tk.Label(self.root, text="加密文本", font=("Arial", 12))
        self.enc_label.grid(row=2, column=2, padx=10, pady=5)
        self.enc_text = tk.Text(self.root, width=40, height=10)
        self.enc_text.grid(row=3, column=2, padx=10, pady=5)

        # 中间按钮（水平对齐）
        self.button_frame = tk.Frame(self.root)
        self.button_frame.grid(row=3, column=1, padx=10, pady=5)

        self.encrypt_btn = tk.Button(self.button_frame, text="-->（加密）", command=self.encrypt_text)
        self.encrypt_btn.pack(pady=5)

        self.decrypt_btn = tk.Button(self.button_frame, text="<--（解密）", command=self.decrypt_text)
        self.decrypt_btn.pack(pady=5)

        # 密钥显示区域
        self.key_frame = tk.Frame(self.root)
        self.key_frame.grid(row=4, column=0, columnspan=3, pady=5)

        self.update_key_fields()

    def encrypt_text(self):
        plaintext = self.src_text.get("1.0", tk.END).strip()
        if not plaintext:
            messagebox.showwarning("警告", "请输入源文本")
            return
        
        mode = self.encryption_mode.get()

        if mode == "AES":
            aes_key = os.urandom(32)
            aes_cipher = AES.new(aes_key, AES.MODE_EAX)
            nonce = aes_cipher.nonce
            ciphertext, tag = aes_cipher.encrypt_and_digest(plaintext.encode())
            
            encrypted_data_json = json.dumps({
                "nonce": base64.b64encode(nonce).decode(),
                "tag": base64.b64encode(tag).decode(),
                "ciphertext": base64.b64encode(ciphertext).decode()
            }, indent=4)

            self.enc_text.delete("1.0", tk.END)
            self.enc_text.insert("1.0", encrypted_data_json)

            self.update_entry(self.aes_key_entry, base64.b64encode(aes_key).decode())

        elif mode == "RSA":
            rsa_cipher = PKCS1_OAEP.new(self.public_key)
            encrypted_data = base64.b64encode(rsa_cipher.encrypt(plaintext.encode())).decode()

            self.enc_text.delete("1.0", tk.END)
            self.enc_text.insert("1.0", encrypted_data)

            self.update_entry(self.rsa_pub_entry, self.public_key.exportKey().decode())
            self.update_entry(self.rsa_priv_entry, self.private_key.exportKey().decode())
            
        else:  # AES+RSA
            aes_key = os.urandom(32)
            aes_cipher = AES.new(aes_key, AES.MODE_EAX)
            nonce = aes_cipher.nonce
            ciphertext, tag = aes_cipher.encrypt_and_digest(plaintext.encode())
            rsa_cipher = PKCS1_OAEP.new(self.public_key)
            encrypted_aes_key = rsa_cipher.encrypt(aes_key)

            encrypted_data_json = json.dumps({
                "encrypted_aes_key": base64.b64encode(encrypted_aes_key).decode(),
                "nonce": base64.b64encode(nonce).decode(),
                "tag": base64.b64encode(tag).decode(),
                "ciphertext": base64.b64encode(ciphertext).decode()
            })

            self.enc_text.delete("1.0", tk.END)
            self.enc_text.insert("1.0", encrypted_data_json)

            self.update_entry(self.rsa_pub_entry, self.public_key.exportKey().decode())
            self.update_entry(self.rsa_priv_entry, self.private_key.exportKey().decode())
            self.update_entry(self.aes_key_entry, base64.b64encode(aes_key).decode())
            self.update_entry(self.enc_aes_key_entry, base64.b64encode(encrypted_aes_key).decode())

        

    def decrypt_text(self):
        encrypted_data = self.enc_text.get("1.0", tk.END).strip()
        if not encrypted_data:
            messagebox.showwarning("警告", "请输入加密文本")
            return
        
        mode = self.encryption_mode.get()

        try:
            if mode == "AES":
                data = json.loads(encrypted_data)

                nonce = base64.b64decode(data["nonce"])
                tag = base64.b64decode(data["tag"])
                ciphertext = base64.b64decode(data["ciphertext"])
                aes_key = base64.b64decode(self.aes_key_entry.get())

                aes_cipher = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
                plaintext = aes_cipher.decrypt_and_verify(ciphertext, tag).decode()

                self.src_text.delete("1.0", tk.END)
                self.src_text.insert("1.0", plaintext)
            elif mode == "RSA":
                rsa_cipher = PKCS1_OAEP.new(self.private_key)
                decrypted_data = rsa_cipher.decrypt(base64.b64decode(encrypted_data)).decode()

                self.src_text.delete("1.0", tk.END)
                self.src_text.insert("1.0", decrypted_data)
            else:
                # 1. 解析 JSON 结构
                data = json.loads(encrypted_data)

                # 2. 提取 Base64 编码的数据并解码
                encrypted_aes_key = base64.b64decode(data["encrypted_aes_key"])
                nonce = base64.b64decode(data["nonce"])
                tag = base64.b64decode(data["tag"])
                ciphertext = base64.b64decode(data["ciphertext"])

                # 3. 使用 RSA 私钥解密 AES 密钥
                rsa_cipher = PKCS1_OAEP.new(self.private_key)
                aes_key = rsa_cipher.decrypt(encrypted_aes_key)

                # 4. 用 AES 解密数据
                aes_cipher = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
                plaintext = aes_cipher.decrypt_and_verify(ciphertext, tag).decode()

                # 5. 显示解密后的文本
                self.src_text.delete("1.0", tk.END)
                self.src_text.insert("1.0", plaintext)
        except Exception as e:
            messagebox.showerror("错误", f"解密失败: {str(e)}")

    def update_key_fields(self, *args):
        for widget in self.key_frame.winfo_children():
            widget.destroy()

        mode = self.encryption_mode.get()

        if mode == "AES":
            self.aes_key_entry = self.create_labeled_entry(self.key_frame, "AES 密钥")

        elif mode == "RSA":
            self.rsa_pub_entry = self.create_labeled_entry(self.key_frame, "RSA 公钥")
            self.rsa_priv_entry = self.create_labeled_entry(self.key_frame, "RSA 私钥")

        else:  # AES+RSA
            self.rsa_pub_entry = self.create_labeled_entry(self.key_frame, "RSA 公钥")
            self.rsa_priv_entry = self.create_labeled_entry(self.key_frame, "RSA 私钥")
            self.aes_key_entry = self.create_labeled_entry(self.key_frame, "AES 密钥")
            self.enc_aes_key_entry = self.create_labeled_entry(self.key_frame, "加密后 AES 密钥")

    def update_entry(self, entry: tk.Entry, value):
        """更新 Entry 文本框的值"""
        entry.config(state="normal")
        entry.delete(0, tk.END)
        entry.insert(0, value)
        entry.config(state="readonly")

    def create_labeled_entry(self, parent, label_text, width=80, font=("Arial", 10)):
        """创建一个带有 Label 和 Entry 的 GUI 组件，并返回 Entry 组件"""
        label = tk.Label(parent, text=label_text, font=("Arial", 12))
        label.pack()
        entry = tk.Entry(parent, width=width, font=font, state="readonly")
        entry.pack()
        return entry


# 运行 GUI
if __name__ == "__main__":
    root = tk.Tk()
    app = SecureTextApp(root)
    root.mainloop()